# Microtask 2



In [89]:
import json
import csv
import datetime
from collections import defaultdict
import pandas as pd

In [94]:
# Here both start year and end year are included
start_year = 2017
end_year = 2018 

quar1_dates = {"start_date": "01-01", "end_date": "03-31"}
quar2_dates = {"start_date": "04-01", "end_date": "06-30"}
quar3_dates = {"start_date": "07-01", "end_date": "09-30"}
quar4_dates = {"start_date": "10-01", "end_date": "12-31"}

old_committers = set()
old_issue_subs = set()
old_pr_subs = set()

In [95]:
class Quarter:
    
    def __init__(self, number, year):
        self.number = number    #.
        self.year = year   
   
        self.num_commits = 0  
        self.num_issues = 0
        self.num_pullrequests = 0
        
        self.new_committers = 0
        self.new_issue_subs = 0
        self.new_pr_subs = 0
        
        self.start_date = ""
        self.end_date = ""
        
        if self.number == 1:
            self.start_date = str(self.year) + '-' + quar1_dates["start_date"]
            self.end_date = str(self.year) + '-' + quar1_dates["end_date"]
            
        if self.number == 2:
            self.start_date = str(self.year) + '-' + quar2_dates["start_date"]
            self.end_date = str(self.year) + '-' + quar3_dates["end_date"]            
            
        if self.number == 3:
            self.start_date = str(self.year) + '-' + quar3_dates["start_date"]
            self.end_date = str(self.year) + '-' + quar3_dates["end_date"]
            
        if self.number == 4:
            self.start_date = str(self.year) + '-' + quar4_dates["start_date"]
            self.end_date = str(self.year) + '-' + quar4_dates["end_date"]
        
            
    def is_includes_data(self, date):
        if self._str_to_dt_quarter(self.start_date) <= self._str_to_dt_data(date) < self._str_to_dt_quarter(self.end_date):
            return True

        return False
    
    def add_analysis(self, datapoint):
        print("hash", datapoint['hash'])
        if datapoint['category'] == "commit":
            self.num_commits += 1 
            
            if datapoint['author'] not in old_committers:
                self.new_committers += 1
                
            old_committers.add(datapoint['author'])
            
                
        if datapoint['category'] == "issue":
            self.num_issues += 1 
            
            if datapoint['author'] not in old_issue_subs:
                self.new_issue_subs += 1
                
            old_issue_subs.add(datapoint['author'])
            
                
        if datapoint['category'] == "pull_request":
            self.num_pullrequests += 1 
            
            if datapoint['author'] not in old_pr_subs:
                self.new_pr_subs += 1
                
            old_pr_subs.add(datapoint['author'])
    
    @staticmethod
    def _str_to_dt_data(date):
        try:
            datetimestr =  datetime.datetime.strptime(date, "%a %b %d %H:%M:%S %Y %z").strftime("%Y-%m-%d")
            
        
        except ValueError as ve:
            datetimestr =  datetime.datetime.strptime(date, "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%d")
        
        finally:
            datetimeobj = datetime.datetime.strptime(datetimestr, "%Y-%m-%d")
            return datetimeobj
    
#     @staticmethod
#     def _str_to_dt_issue(date):
        
#         datetimestr =  datetime.datetime.strptime(date, "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%d")
#         datetimeobj = datetime.datetime.strptime(datetimestr, "%Y-%m-%d")
#         return datetimeobj
    
        
    @staticmethod
    def _str_to_dt_quarter(date):
        
        datetimeobj =  datetime.datetime.strptime(date, "%Y-%m-%d")
        return datetimeobj
    
    def __str__(self):
        return str(self.number) + " " + str(self.year)

In [121]:
class CleanJson():   
    
    def __init__(self, path_to_file):

        clean_commit_list = list()
        clean_issue_list = list()
        clean_pr_list = list()
        
        with open(path_to_file, 'r') as raw_data:
            for line in raw_data:
                line = json.loads(line)

#                 print(line)
                clean_line = dict()
                if line['category'] == "commit":
                    clean_line = self._clean_commit(line)
                    clean_commit_list.append(clean_line)
                    

                elif line['category'] == "issue":
                    clean_line = self._clean_issue(line)
                    clean_issue_list.append(clean_line)
                    

                elif line['category'] == "pull_request":
                    clean_line = self._clean_pr(line)
                    clean_pr_list.append(clean_line)
                        
                        
                self.clean_commit_df = pd.DataFrame(clean_commit_list)
                self.clean_issue_df = pd.DataFrame(clean_issue_list)
                self.clean_pr_df = pd.DataFrame(clean_pr_list)
                
            self.clean_dict = {
                'commit': self.clean_commit_df,
                'issue': self.clean_issue_df,
                'pull_request': self.clean_pr_df
            }

    @staticmethod
    def _clean_commit(line):
            repo_name = line['origin']
            line_data = line['data']
            cleaned_line = {
                'repo': repo_name,
                'hash': line_data['commit'],
                'category': "commit",
                'commit': line_data['Commit'],
                'author': line_data['Author'],
                'created_date': line_data['CommitDate'],
                'files_no': len(line_data['files'])
            }

            actions = 0

            for file in line_data['files']:
                if 'action' in file:
                    actions += 1
                    cleaned_line['files_action'] = actions
                    cleaned_line['merge'] = 'Merge' in line_data
            return cleaned_line

    @staticmethod
    def _clean_issue(line):
        repo_name = line['origin']
        line_data = line['data']
        cleaned_line ={
            'repo': repo_name,
            'hash': line_data['id'],
            'category': "issue",
            'author': line_data['user']['login'],
            'created_date': line_data['created_at'],
            'current_status': line_data['state']   
        }

        return cleaned_line

    @staticmethod
    def _clean_pr(line):
        repo_name = line['origin']
        line_data = line['data']
        cleaned_line ={
            'repo': repo_name,
            'hash': line_data['id'],
            'category': "pull_request",
            'author': line_data['user']['login'],
            'created_date': line_data['created_at'],
            'current_status': line_data['state']   
        }

        return cleaned_line
    

In [122]:
year_list = [x for x in range(start_year, end_year + 1)]
quar_list = [Quarter(num, year)  for year in year_list for num in range(1, 5)]
!wc -l ../../progit.json

4066 ../../progit.json


In [123]:
clean_data = CleanJson('../../progit.json')

In [124]:
print(clean_data.clean_list[0])

                                                 author category  \
0                      Scott Chacon <schacon@gmail.com>   commit   
1                      Scott Chacon <schacon@gmail.com>   commit   
2                      Scott Chacon <schacon@gmail.com>   commit   
3                      Scott Chacon <schacon@gmail.com>   commit   
4                            Ben Straub <bs@github.com>   commit   
5                            Ben Straub <bs@github.com>   commit   
6                            Ben Straub <bs@github.com>   commit   
7                            Ben Straub <bs@github.com>   commit   
8                            Ben Straub <bs@github.com>   commit   
9                            Ben Straub <bs@github.com>   commit   
10                           Ben Straub <bs@github.com>   commit   
11                           Ben Straub <bs@github.com>   commit   
12                           Ben Straub <bs@github.com>   commit   
13                           Ben Straub <bs@gith

In [125]:
for df in (clean_data.clean_list):
    print(df.loc[0]['category'])
    data = df
    for quarter in quar_list:
        print(".. .. ..")
        for index, data_point in data.iterrows():
#             print(type(data_point))
#             print(data_point)
            print("././././")
            data_point = pd.Series.to_dict(data_point)
            print(data_point["created_date"])
            if quarter.is_includes_data(data_point["created_date"]):
                quarter.add_analysis(data_point)


commit
.. .. ..
././././
Thu Jan 2 17:23:47 2014 -0800
././././
Thu Jan 2 17:39:50 2014 -0800
././././
Thu Jan 2 17:41:15 2014 -0800
././././
Thu Jan 2 17:41:33 2014 -0800
././././
Mon Jan 6 14:17:35 2014 -0800
././././
Mon Jan 6 14:28:26 2014 -0800
././././
Tue Jan 7 11:24:29 2014 -0800
././././
Tue Jan 7 11:35:24 2014 -0800
././././
Tue Jan 7 15:48:31 2014 -0800
././././
Wed Jan 8 15:21:02 2014 -0800
././././
Wed Jan 8 15:37:54 2014 -0800
././././
Thu Jan 9 08:46:48 2014 -0800
././././
Thu Jan 9 08:58:43 2014 -0800
././././
Sun Jan 12 16:20:56 2014 -0800
././././
Sun Jan 12 16:21:06 2014 -0800
././././
Sun Jan 12 16:21:14 2014 -0800
././././
Wed Jan 22 06:05:42 2014 -0800
././././
Wed Jan 22 07:04:41 2014 -0800
././././
Wed Jan 22 07:05:31 2014 -0800
././././
Wed Jan 22 07:36:01 2014 -0800
././././
Wed Jan 22 07:48:28 2014 -0800
././././
Wed Jan 22 07:48:35 2014 -0800
././././
Thu Jan 30 05:38:18 2014 -0800
././././
Thu Jan 30 05:42:01 2014 -0800
././././
Thu Jan 30 05:42:15 2014 -08

Mon Jun 9 19:37:14 2014 -0700
././././
Mon Jun 9 20:12:00 2014 -0700
././././
Mon Jun 9 20:27:11 2014 -0700
././././
Thu Jun 12 13:04:15 2014 -0700
././././
Thu Jun 12 15:54:28 2014 -0700
././././
Thu Jun 12 15:54:30 2014 -0700
././././
Fri Jun 13 07:22:17 2014 -0700
././././
Fri Jun 13 07:24:27 2014 -0700
././././
Fri Jun 13 07:25:05 2014 -0700
././././
Thu Jun 12 11:52:17 2014 -0700
././././
Fri Jun 13 07:52:58 2014 -0700
././././
Fri Jun 13 09:05:05 2014 -0700
././././
Fri Jun 13 09:06:07 2014 -0700
././././
Fri Jun 13 09:09:49 2014 -0700
././././
Sat Jul 5 10:50:17 2014 -0600
././././
Wed Jul 9 20:54:54 2014 -0700
././././
Wed Jul 9 21:20:17 2014 -0700
././././
Wed Jul 9 21:44:23 2014 -0700
././././
Mon Jul 28 12:30:45 2014 -0700
././././
Mon Jul 28 12:40:11 2014 -0700
././././
Mon Jul 28 15:14:39 2014 -0700
././././
Thu Jun 12 11:05:21 2014 -0700
././././
Thu Jun 12 12:57:46 2014 -0700
././././
Thu Jul 10 20:45:06 2014 -0700
././././
Thu Jul 10 20:49:09 2014 -0700
././././
Thu Jul

Tue Sep 23 13:00:35 2014 +0200
././././
Tue Sep 23 13:08:48 2014 +0200
././././
Tue Sep 23 13:18:12 2014 +0200
././././
Tue Sep 23 13:19:15 2014 +0200
././././
Tue Sep 23 13:20:44 2014 +0200
././././
Tue Sep 23 13:41:48 2014 +0200
././././
Tue Sep 23 13:42:01 2014 +0200
././././
Tue Sep 23 16:12:00 2014 +0200
././././
Tue Sep 23 16:12:31 2014 +0200
././././
Tue Sep 23 16:23:06 2014 +0200
././././
Tue Sep 23 16:43:41 2014 +0200
././././
Tue Sep 23 16:43:51 2014 +0200
././././
Tue Sep 23 16:47:01 2014 +0200
././././
Tue Sep 23 16:54:44 2014 +0200
././././
Wed Sep 24 09:27:21 2014 +0200
././././
Wed Sep 24 10:14:15 2014 +0200
././././
Wed Sep 24 10:16:50 2014 +0200
././././
Wed Sep 24 12:14:31 2014 +0200
././././
Tue Sep 30 15:57:25 2014 +0200
././././
Wed Sep 24 12:15:06 2014 +0200
././././
Wed Sep 24 12:15:07 2014 +0200
././././
Wed Sep 24 12:15:07 2014 +0200
././././
Wed Sep 24 12:15:07 2014 +0200
././././
Wed Sep 24 12:15:07 2014 +0200
././././
Mon Sep 29 15:40:30 2014 +0200
././././


././././
Tue Oct 28 00:31:10 2014 +0800
././././
Mon Oct 27 20:00:38 2014 +0300
././././
Mon Oct 27 20:04:12 2014 +0300
././././
Mon Oct 27 22:14:48 2014 +0300
././././
Mon Oct 27 22:19:24 2014 +0400
././././
Mon Oct 27 22:22:49 2014 +0400
././././
Mon Oct 27 22:43:16 2014 +0400
././././
Tue Oct 28 00:14:45 2014 +0300
././././
Tue Oct 28 14:29:54 2014 +0300
././././
Fri Oct 31 10:29:26 2014 +0300
././././
Wed Oct 29 00:54:41 2014 +0300
././././
Wed Oct 29 00:57:19 2014 +0300
././././
Wed Oct 29 11:43:40 2014 +0100
././././
Thu Oct 30 14:14:58 2014 +0400
././././
Thu Oct 30 19:38:38 2014 +0400
././././
Thu Oct 30 23:20:12 2014 +0300
././././
Thu Oct 30 23:36:41 2014 +0300
././././
Thu Oct 30 23:44:23 2014 +0300
././././
Fri Oct 31 15:36:48 2014 +0300
././././
Fri Oct 31 19:40:38 2014 +0300
././././
Sat Nov 1 01:02:16 2014 +0300
././././
Sun Nov 2 01:10:37 2014 +0300
././././
Sun Nov 2 07:59:35 2014 +0300
././././
Sun Nov 2 12:22:41 2014 +0800
././././
Sun Nov 2 13:26:45 2014 +0300
././.

././././
Tue Dec 16 01:02:46 2014 +0300
././././
Tue Dec 16 10:08:57 2014 +0300
././././
Fri Dec 12 23:49:23 2014 +0600
././././
Mon Dec 15 23:21:33 2014 +0600
././././
Tue Dec 16 10:09:47 2014 +0300
././././
Sun Dec 7 14:03:58 2014 +0300
././././
Sun Dec 7 14:08:11 2014 +0300
././././
Fri Dec 12 01:28:47 2014 +0300
././././
Fri Dec 12 01:30:31 2014 +0300
././././
Mon Dec 15 13:15:32 2014 +0300
././././
Tue Dec 16 10:10:18 2014 +0300
././././
Fri Dec 19 14:42:29 2014 +0300
././././
Fri Dec 19 17:16:43 2014 +0300
././././
Thu Dec 18 22:02:50 2014 +0300
././././
Fri Dec 19 17:16:59 2014 +0300
././././
Tue Dec 23 21:05:45 2014 +0300
././././
Wed Dec 24 12:22:53 2014 +0300
././././
Sun Dec 21 12:51:59 2014 +0300
././././
Wed Dec 24 12:23:10 2014 +0300
././././
Sat Dec 20 15:07:22 2014 +0300
././././
Wed Dec 24 12:23:22 2014 +0300
././././
Wed Dec 17 15:06:10 2014 +0300
././././
Wed Dec 17 15:10:22 2014 +0300
././././
Wed Dec 24 12:23:48 2014 +0300
././././
Tue Dec 16 18:45:51 2014 +0300
./

...

././././
2019-02-18T08:05:28Z
././././
2019-02-11T21:42:23Z
././././
2019-02-18T08:12:25Z
././././
2019-02-15T12:19:22Z
././././
2019-02-22T08:19:07Z
././././
2019-02-22T11:42:45Z
././././
2019-02-22T10:39:25Z
././././
2019-02-22T12:05:57Z
././././
2019-02-22T12:13:34Z
././././
2019-02-22T12:11:16Z
././././
2019-02-22T12:09:38Z
././././
2019-02-19T07:35:04Z
././././
2019-02-19T07:23:55Z
././././
2014-11-01T12:31:00Z
././././
2014-11-01T14:11:06Z
././././
2014-11-04T17:37:49Z
././././
2014-11-02T06:35:35Z
././././
2014-11-05T04:39:38Z
././././
2014-11-05T20:00:46Z
././././
2014-11-07T16:13:38Z
././././
2014-11-13T14:34:25Z
././././
2014-11-07T22:25:56Z
././././
2014-11-12T05:03:22Z
././././
2014-12-09T07:52:54Z
././././
2014-12-11T16:32:17Z
././././
2014-11-12T09:14:48Z
././././
2014-12-28T15:49:55Z
././././
2015-01-14T15:33:16Z
././././
2014-12-30T02:33:14Z
././././
2015-02-08T11:02:50Z
././././
2015-02-16T09:26:20Z
././././
2015-02-16T08:35:54Z
././././
2015-02-08T11:14:49Z
././././
2

2014-10-30T11:15:50Z
././././
2014-10-29T10:55:48Z
././././
2014-10-29T10:55:33Z
././././
2014-10-29T10:54:55Z
././././
2014-10-29T10:39:57Z
././././
2014-10-29T22:03:31Z
././././
2014-10-30T22:50:48Z
././././
2014-11-02T04:25:30Z
././././
2014-10-31T22:06:39Z
././././
2014-11-01T11:51:48Z
././././
2014-11-02T09:51:59Z
././././
2014-11-02T16:58:48Z
././././
2014-10-30T00:15:17Z
././././
2014-11-05T07:55:04Z
././././
2014-11-05T14:47:15Z
././././
2014-11-02T20:09:02Z
././././
2014-11-06T10:26:32Z
././././
2014-11-06T15:10:04Z
././././
2014-11-07T13:08:13Z
././././
2014-11-09T15:48:49Z
././././
2014-11-10T00:20:34Z
././././
2014-11-09T23:00:46Z
././././
2014-11-09T21:15:18Z
././././
2014-11-12T22:13:22Z
././././
2014-11-10T21:52:51Z
././././
2014-11-13T23:25:46Z
././././
2014-11-14T22:58:38Z
././././
2014-11-11T15:58:24Z
././././
2014-11-21T21:49:26Z
././././
2014-11-22T13:28:47Z
././././
2014-11-13T02:22:22Z
././././
2014-11-16T16:29:06Z
././././
2014-11-13T02:22:59Z
././././
2014-11-23

././././
2017-06-09T09:28:03Z
././././
2017-08-27T01:54:02Z
././././
2017-08-27T09:31:59Z
././././
2017-08-27T09:41:55Z
././././
2017-09-20T13:54:38Z
././././
2017-10-23T13:19:39Z
././././
2017-10-23T16:22:39Z
././././
2017-10-22T09:55:27Z
././././
2017-10-22T09:47:33Z
././././
2017-10-24T13:05:01Z
././././
2018-01-01T06:48:46Z
././././
2018-01-15T10:27:20Z
././././
2018-01-16T04:44:22Z
././././
2018-02-19T09:49:43Z
././././
2018-04-15T05:45:28Z
hash 181682208
././././
2018-04-29T10:29:03Z
hash 184828061
././././
2018-04-29T16:21:16Z
hash 184842302
././././
2018-04-29T15:13:36Z
hash 184839201
././././
2018-05-10T10:11:12Z
hash 187152666
././././
2018-05-19T10:17:55Z
hash 189181782
././././
2018-07-27T06:27:00Z
hash 204340463
././././
2018-05-08T08:01:43Z
hash 186555659
././././
2018-09-11T09:20:16Z
hash 214565857
././././
2018-09-20T02:56:07Z
hash 216830671
././././
2019-01-25T02:22:40Z
././././
2019-01-22T07:51:29Z
././././
2019-03-11T00:38:16Z
.. .. ..
././././
2014-10-30T11:15:50Z
.

2015-07-06T06:10:40Z
././././
2015-07-06T06:26:23Z
././././
2015-07-06T06:31:02Z
././././
2015-07-06T06:35:43Z
././././
2015-07-06T06:53:22Z
././././
2015-08-11T04:23:35Z
././././
2015-07-06T07:05:20Z
././././
2015-08-16T09:37:52Z
././././
2015-06-19T13:11:06Z
././././
2015-08-13T11:15:21Z
././././
2015-06-25T09:20:43Z
././././
2015-06-24T10:18:57Z
././././
2015-08-16T16:21:47Z
././././
2015-08-17T01:13:56Z
././././
2015-08-17T10:36:23Z
././././
2015-06-19T12:21:28Z
././././
2015-08-17T14:05:52Z
././././
2015-08-17T15:24:56Z
././././
2015-08-17T15:12:46Z
././././
2015-08-17T23:41:22Z
././././
2015-08-16T19:02:34Z
././././
2015-08-18T00:30:55Z
././././
2015-08-18T01:32:48Z
././././
2015-08-18T02:09:00Z
././././
2015-08-17T12:31:20Z
././././
2015-08-17T13:56:16Z
././././
2015-08-18T08:15:44Z
././././
2015-08-18T09:53:22Z
././././
2015-08-18T07:39:35Z
././././
2015-08-18T06:43:36Z
././././
2015-08-14T08:57:01Z
././././
2015-08-18T04:34:15Z
././././
2015-08-18T03:06:48Z
././././
2015-08-19

././././
2015-01-14T15:33:16Z
././././
2014-12-30T02:33:14Z
././././
2015-02-08T11:02:50Z
././././
2015-02-16T09:26:20Z
././././
2015-02-16T08:35:54Z
././././
2015-02-08T11:14:49Z
././././
2015-02-17T05:02:39Z
././././
2015-02-17T10:21:12Z
././././
2015-05-09T17:33:14Z
././././
2015-02-16T08:52:09Z
././././
2015-02-17T11:04:54Z
././././
2015-02-22T10:59:09Z
././././
2015-05-29T08:46:57Z
././././
2015-05-29T16:20:03Z
././././
2015-05-29T16:41:48Z
././././
2015-02-26T12:50:39Z
././././
2015-06-03T06:03:26Z
././././
2015-06-05T11:22:23Z
././././
2015-06-08T04:17:25Z
././././
2015-06-09T14:41:32Z
././././
2015-06-10T07:10:19Z
././././
2015-06-09T18:21:09Z
././././
2015-06-13T02:04:28Z
././././
2015-06-12T08:58:13Z
././././
2015-06-16T03:39:47Z
././././
2015-06-14T17:43:49Z
././././
2015-06-16T07:04:36Z
././././
2015-06-16T07:09:22Z
././././
2015-06-15T16:00:56Z
././././
2015-06-16T07:44:05Z
././././
2015-06-16T08:26:32Z
././././
2015-06-16T14:29:54Z
././././
2015-06-17T02:20:47Z
././././
2

# Analysis



## Number of commits, pull requests and issues per quadrant

In [126]:
for q in quar_list:
    print(q, q.num_commits)
    
for i in old_committers:
    print(i)

1 2017 13
2 2017 9
3 2017 5
4 2017 11
1 2018 18
2 2018 19
3 2018 3
4 2018 4
Andre Polykanine A.K.A. Menelion Elensúlë <andre@oire.org>
chstarkd <43615579+chstarkd@users.noreply.github.com>
Eplehova <32541719+Eplehova@users.noreply.github.com>
Leonid Kozarin <kozalo@sadbot.ru>
晓荷 <xhlsrj@qq.com>
Anton Tsyganenko <anton-tsyganenko@yandex.ru>
Bogdan <BogdanDor@users.noreply.github.com>
foobar47 <someone@nowhere.com>
marjune <marjune@163.com>
Matveev, Sergey <sergey.matveev@intel.com>
靳阳 <260893248@qq.com>
Denis Savitskiy <sadfuzzy@users.noreply.github.com>
networm <zzjbcn@gmail.com>
ljporljp <962696057@qq.com>
ioannkozlitin <ioannkozlitin@gmail.com>
Grigory Tumakov <tumakov@morizo.ru>
Jean-Noel Avila <jn.avila@free.fr>
Jean-Noël Avila <jn.avila@free.fr>
chen <kikyous@163.com>
Gerry <hgr3306@foxmail.com>
CaoWeiwei <icaoweiwei@users.noreply.github.com>
Anton Petrunin <petrunin@math.psu.edu>
狂飙 <zzjbcn@gmail.com>
Andre Polykanine <ap@oire.me>
wangfpp <35289556+wangfpp@users.noreply.github.co

## Number of new committers, new issue submitters and pull request creators

In [127]:
for q in quar_list:
    print(q, q.new_committers)

1 2017 7
2 2017 5
3 2017 0
4 2017 4
1 2018 8
2 2018 6
3 2018 0
4 2018 1


# Viewing data as a csv file

## Writing the cleaned data to a csv

In [128]:
def write_to_csv(file_path):
    with open(file_path, 'w', ) as csvfile:
        csv_writer = csv.writer(csvfile, delimiter=',')
        
        file_headers = ["Quarter(Num)", "Quarter(Year)", "Num_Commits", "Num_Issues", "Num_PRs", "Num_new_commits", "Num_new_issues", "Num_new_prs"]
        csv_writer.writerow(file_headers)
        
        for quar in quar_list:
            row = [str(quar.number),       \
                 str(quar.year),         \
                 str(quar.num_commits) ,     \
                 str(quar.num_issues)  ,    \
                 str(quar.num_pullrequests), \
                 str(quar.new_committers),  \
                 str(quar.new_issue_subs), \
                 str(quar.new_pr_subs)       
                  ]
            csv_writer.writerow(x for x in row)
            

In [132]:
write_to_csv("./progit.csv")

## Displaying a table based on the csv file

In [135]:
pd.read_csv('./progit.csv')

,Quarter(Num),Quarter(Year),Num_Commits,Num_Issues,Num_PRs,Num_new_commits,Num_new_issues,Num_new_prs
0,1,2017,13,8,4,7,7,4
1,2,2017,9,12,8,5,9,7
2,3,2017,5,8,5,0,0,0
3,4,2017,11,8,7,4,4,3
4,1,2018,18,12,7,8,9,6
5,2,2018,19,16,12,6,9,8
6,3,2018,3,4,4,0,0,0
7,4,2018,4,6,2,1,5,1
